# 02 — RNN pooling fix (SmartWatch Gestures)

# Doel
De baseline gebruikt een naïeve sequentierepresentatie door alleen de **laatste timestep** van de (gepaddede) sequentie te gebruiken voor classificatie.  
Omdat sequenties **variabele lengte** hebben en op batch-niveau worden **gepaddede**, kan deze keuze informatieverlies veroorzaken of padding-effecten introduceren.

# Hypothese
Als we de sequentierepresentatie vervangen door **pooling over tijd** (bijv. gemiddelde over alle timesteps), dan wordt het model robuuster voor variabele sequentielengtes en verwachten we een **hogere en/of stabielere validatie-accuracy**.

# Experiment
We houden dataset, preprocessor, TrainerSettings en optimizer gelijk, en vergelijken:
- Baseline representatie: laatste timestep
- Pooling representatie: mean pooling over tijd

Resultaten worden gelogd in MLflow zodat runs direct vergelijkbaar zijn.


In [ ]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

In [ ]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Override naar CPU voor kleine RNN-modellen indien gewenst
device = torch.device("cpu")

device


In [ ]:
# Data + preprocessor (identiek aan notebook 01)
preprocessor = PaddedPreprocessor()

gestures_factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gestures_factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

len(train), len(valid)

In [ ]:
accuracy = Accuracy()

settings = TrainerSettings(
    epochs=30,  # bewust korter dan "100": dit notebook is een gerichte vergelijking
    metrics=[accuracy],
    logdir=Path("gestures_ex3"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
        "delta": 0.0,
    },
)
settings

In [ ]:
from src_rnn.models import RNNConfig
from src_rnn.run_experiment import run_experiment

In [ ]:
# Run 1: Baseline representatie (laatste timestep)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_last",                 # baseline: last-step representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_baseline_gru_last_h64_l1",
)


In [ ]:
# Run 2: Pooling fix (mean pooling over tijd)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_mean",                 # fix: mean-pooling representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_fix_gru_meanpool_h64_l1",
)